# Project 1: Recipe Assistant

In the Workflow Examples chapter, you built a **fixed recipe pipeline** — three LLM calls chained together in a strict order. You controlled every step.

Now let's build an **agent** that searches for real recipes on the internet, lets you review the result in a **form**, and — if you choose — generates a shopping list with cost estimates. It uses a **chat interface** with **memory**, so you can ask follow-up questions.

| What you'll learn | Where it comes from |
|---|---|
| **HTTP Request Tool** — fetch live data from APIs | New concept |
| **Fallback strategy** — two tools for the same task | Builds on the Tool Calling chapter |
| **Chat Trigger + Memory** — multi-turn conversation | Builds on the First AI Agent chapter |
| **Wait node with Form** — human reviews before continuing | Builds on Human-in-the-Loop |
| **IF branching** — different paths based on user choice | Builds on Routing pattern |

---

## The Workflow

```
STAGE 1: Find Recipe              STAGE 2: Review        STAGE 3: Shopping List (optional)
─────────────────────             ──────────────         ─────────────────────────────────
┌──────────┐  ┌─────────────┐  ┌──────────┐  ┌───────────┐  ┌──────────────┐  ┌─────────────────┐
│ Chat     │─▶│ Recipe      │─▶│ Review   │─▶│ Want      │─▶│ Shopping     │─▶│ Output — With   │
│ Trigger  │  │ Search Agent│  │ Recipe   │  │ Prices?   │  │ List LLM     │  │ Prices          │
└──────────┘  └─────────────┘  │ (Form)   │  │ (IF)      │  └──────────────┘  └─────────────────┘
                    ┊          └──────────┘  │           │
         ┌──────────┼──────────┐             │  No ──────│─▶┌─────────────────┐
         ┊          ┊     ┊    ┊             └───────────┘  │ Output — Recipe  │
    Chat Model  Memory  HTTP  SerpAPI                       │ Only             │
                      (MealDB) (fallback)                   └─────────────────┘
```

The user types in a **chat window**. The agent searches for recipes (with memory for follow-ups), then the workflow pauses at a **form** where you review the recipe and choose whether to generate a shopping list.

---

**File:** [`10_recipe_assistant.json`](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/10_recipe_assistant.json)

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/10_recipe_assistant.json
> ```
>
> **Download:** {download}`10_recipe_assistant.json <_static/workflows/10_recipe_assistant.json>`

**Credentials needed:** OpenRouter API key + SerpAPI key (both from earlier chapters).

---

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Recipe Assistant"

### Step 2: Add the Chat Trigger

1. The **When chat message received** node should already be on the canvas (n8n adds it by default for new workflows). If not, add it manually.
2. No configuration needed — it works out of the box.

### Step 3: Add the AI Agent node

1. Add **AI Agent** → rename to `Recipe Search Agent`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression): `{{ $json.chatInput }}`
   - **System Message** (in Options):
     ```
     You are a friendly recipe assistant that helps users find recipes.

     Searching for recipes:
     1. Convert the user's request to a simple English search term for the tool (e.g. "quiero pollo" → search for "chicken", "pasta con tomate" → search for "tomato pasta")
     2. Use Recipe Search (TheMealDB) first — it returns structured, reliable data
     3. Only use Google Search if TheMealDB returns no results (meals is null)

     Presenting recipes — use this format:
     RECIPE: [name]
     Category: [category] | Cuisine: [cuisine]

     INGREDIENTS:
     - [quantity] [ingredient]
     ...

     INSTRUCTIONS:
     1. [step]
     ...

     Rules:
     - Never invent recipes — only present what you find from tools
     - Include ALL ingredients with exact quantities from the source
     - If no results, suggest alternative search terms
     ```

### Step 4: Add the Chat Model (sub-node)

1. Click **+ Chat Model** at the bottom of the AI Agent node
2. Select **OpenRouter Chat Model**
3. Choose your credential
4. Model: `openai/gpt-4o-mini`

### Step 5: Add the Memory (sub-node)

1. Click **+ Memory** at the bottom of the AI Agent node
2. Select **Window Buffer Memory**
3. Configure:
   - **Context Window Length**: `10` (remembers last 10 messages)
   - **Session ID**: Custom Key with `{{ $json.sessionId }}`

### Step 6: Add the HTTP Request Tool (sub-node)

1. Click **+ Tool** at the bottom of the AI Agent node
2. Select **HTTP Request Tool**
3. Rename to `Recipe Search`
4. Configure:

| Setting | Value |
|---------|-------|
| **Name** | `Recipe Search` |
| **Description** | `Search for recipes by name or ingredient on TheMealDB. Input should be a simple English search term like 'chicken' or 'pasta'. Returns recipe details including name, ingredients with quantities, and cooking instructions. If the response contains meals:null, it means no recipes were found — try Google Search instead.` |
| **Method** | `GET` |
| **URL** | `https://www.themealdb.com/api/json/v1/1/search.php` |

5. Toggle ON **Send Query Parameters** and add one parameter:

| Key | Value |
|-----|-------|
| `s` | Click the **✨ button** next to the value field (see note below) |

6. After clicking ✨, n8n creates a `$fromAI()` expression. Edit the description to:
   ```
   The recipe search term in English, a simple food keyword like chicken or pasta
   ```
7. Enable **Optimize Response** to reduce token usage

```{tip}
**The ✨ button:** Hover over the value field of a query parameter — a small sparkle icon (✨) appears. Clicking it tells n8n: "let the AI fill this value." n8n generates a `$fromAI()` expression that the agent fills at runtime.

If you prefer to type the expression manually, switch the value field to **Expression** mode and enter:

    {{ $fromAI('query', 'The recipe search term in English, a simple food keyword like chicken or pasta', 'string') }}
```

### Step 7: Add the SerpAPI Tool (sub-node)

1. Click **+ Tool** at the bottom of the AI Agent node
2. Select **SerpAPI** (you should already have credentials from the First AI Agent chapter)
3. Rename to `Google Search`

### Step 8: Add the Wait node (Form)

1. Add **Wait** → rename to `Review Recipe`
2. Configure:
   - **Resume**: `On Form Submitted`
   - **Form Title**: `Recipe Found`
   - **Form Description**: `The agent found a recipe. Review it below and choose whether to generate a shopping list with price estimates.`
   - **Button Label** (in Options): `Continue`
3. Add two form fields:

**Field 1 — Recipe display (read-only):**

| Setting | Value |
|---------|-------|
| **Type** | `Custom HTML` |
| **Label** | `Recipe` |
| **HTML** (Expression) | `<pre style='white-space:pre-wrap;max-height:500px;overflow-y:auto;background:#f8f9fa;padding:16px;border-radius:8px;font-family:system-ui,sans-serif;font-size:14px;line-height:1.6;border:1px solid #dee2e6;'>{{ $json.output }}</pre>` |

**Field 2 — User decision:**

| Setting | Value |
|---------|-------|
| **Type** | `Dropdown` |
| **Label** | `Generate shopping list?` |
| **Required** | Yes |
| **Options** | `Yes, with price estimates` and `No, just the recipe` |

### Step 9: Add the IF node

1. Add **IF** → rename to `Want Prices?`
2. Add condition:
   - **Left Value** (Expression): `{{ $json["Generate shopping list?"] }}`
   - **Operation**: `equals`
   - **Right Value**: `Yes, with price estimates`

### Step 10: Add the Shopping List LLM (true branch)

1. From the **true** output of the IF node, add **Basic LLM Chain** → rename to `Shopping List LLM`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression): `{{ $('Recipe Search Agent').first().json.output }}`
   - **System Message**:
     ```
     You are a shopping assistant.
     From the recipe provided, create a complete shopping list.

     Format:
     ## Shopping List

     ### Produce
     - [quantity] [item] — ~$[price]

     ### Meat / Fish
     - [quantity] [item] — ~$[price]

     ### Dairy
     - [quantity] [item] — ~$[price]

     ### Pantry
     - [quantity] [item] — ~$[price]

     ---
     **Estimated Total: ~$[total]**

     Budget Tips:
     - [tip 1]
     - [tip 2]

     Rules:
     - Include exact quantities from the recipe
     - Skip items most kitchens already have (salt, pepper, cooking oil)
     - Use realistic US grocery store prices (2024 averages)
     - Group items by store section
     - Output ONLY the shopping list — no recipe repetition
     ```
3. Add **Chat Model** sub-node (OpenRouter, `openai/gpt-4o-mini`)

### Step 11: Add Output nodes

**Output — With Prices** (after Shopping List LLM):
1. Add **Edit Fields** → rename to `Output — With Prices`
2. Add one String field:
   - `output` (Expression): `{{ $('Recipe Search Agent').first().json.output + '\n\n---\n\n' + $json.text }}`

**Output — Recipe Only** (from the **false** output of the IF node):
1. Add **Edit Fields** → rename to `Output — Recipe Only`
2. Add one String field:
   - `output` (Expression): `{{ $('Recipe Search Agent').first().json.output }}`

### Step 12: Connect the main nodes

```
Chat Trigger → Recipe Search Agent → Review Recipe → Want Prices?
    Want Prices? (true)  → Shopping List LLM → Output — With Prices
    Want Prices? (false) → Output — Recipe Only
```

::::

## The Agent

### What Is an API Endpoint?

An **API endpoint** is a specific URL that returns data when you call it. Think of it as a vending machine: you press a button (the URL + parameters), and it gives you a specific product (JSON data).

```
You (n8n)                          API Server
─────────                          ──────────
  GET /search?s=chicken   ──────▶  TheMealDB
                          ◀──────  { meals: [...] }
```

The **HTTP Request Tool** in n8n lets an agent call any API endpoint.

### TheMealDB: A Free Recipe API

[TheMealDB](https://www.themealdb.com/api.php) is a free recipe database. No API key needed — the test key `1` is built into the URL.

| Endpoint | URL | What it returns |
|----------|-----|----------------|
| **Search by name** | `.../search.php?s=chicken` | Full recipes matching the term |
| **Random recipe** | `.../random.php` | One random recipe |
| **Filter by category** | `.../filter.php?c=Seafood` | Recipe names in a category |
| **Filter by cuisine** | `.../filter.php?a=Italian` | Recipe names from a cuisine |

All URLs start with `https://www.themealdb.com/api/json/v1/1/`.

**Key details:**
- Results come inside a `"meals"` array
- If nothing is found, `"meals"` is `null` (not an empty array)
- Ingredients and measures are stored as 20 numbered pairs (`strIngredient1`/`strMeasure1` through `strIngredient20`/`strMeasure20`)

### Two Tools, One Strategy

The agent has **two search tools** — a primary and a fallback:

| Tool | Type | When the agent uses it |
|------|------|----------------------|
| **Recipe Search** | HTTP Request → TheMealDB | First choice — structured data, free, no key |
| **Google Search** | SerpAPI | Fallback — if TheMealDB has no results |

TheMealDB has ~300 recipes with clean, structured JSON. But it doesn't have every recipe. SerpAPI searches Google and can find any recipe, but the results are unstructured web pages.

The system message tells the agent: **"Always try TheMealDB first. Only use Google if TheMealDB returns no results."** This is a **fallback strategy** — a common pattern in production systems.

### HTTP Request Tool with `$fromAI()`

Until now, your agents used built-in tools (Calculator, Wikipedia, SerpAPI). The **HTTP Request Tool** lets an agent call any URL — opening the door to thousands of public APIs.

The agent needs to fill in the search term dynamically. We use `$fromAI()` — a special function that tells the AI model: "you need to provide a value for this parameter."

In the HTTP Request Tool, we set:
- **URL**: `https://www.themealdb.com/api/json/v1/1/search.php` (static, no query string)
- **Send Query Parameters**: ON
- **Key**: `s` — **Value**: `$fromAI('query', 'The recipe search term in English...', 'string')`

When the user asks "I want chicken", the agent fills in the value and the tool calls:
```
GET https://www.themealdb.com/api/json/v1/1/search.php?s=chicken
```

#### How to set up `$fromAI()` in n8n

There are two ways:

1. **The ✨ button** (easiest): Hover over a parameter's value field and click the sparkle icon (✨). n8n generates the `$fromAI()` expression for you. Then edit the description so the agent knows what to provide.

2. **Manual expression**: Switch the value field to Expression mode and type:
   ```
   {{ $fromAI('query', 'The recipe search term in English, a simple food keyword like chicken or pasta', 'string') }}
   ```

The `$fromAI()` function takes three arguments:

| Argument | Meaning | Example |
|----------|---------|---------|
| `key` | Parameter name (the agent sees this) | `'query'` |
| `description` | What value to provide | `'A simple food keyword like chicken or pasta'` |
| `type` | Data type | `'string'` |

```{tip}
The system prompt tells the agent to **translate the user's request to English** before searching (e.g. "quiero pollo" → `chicken`). This ensures TheMealDB gets a valid search term regardless of what language the user types in.
```

**Docs:** [HTTP Request Tool](https://docs.n8n.io/integrations/builtin/cluster-nodes/sub-nodes/n8n-nodes-langchain.toolhttprequest/)

### Memory

The **Window Buffer Memory** node stores the last 10 messages of the conversation. This is what lets you have follow-up conversations in the chat.

Without memory, each message would be treated as a completely new conversation.

### System Message

```
You are a friendly recipe assistant that helps users find recipes.

Searching for recipes:
1. Convert the user's request to a simple English search term for the tool (e.g. "quiero pollo" → search for "chicken", "pasta con tomate" → search for "tomato pasta")
2. Use Recipe Search (TheMealDB) first — it returns structured, reliable data
3. Only use Google Search if TheMealDB returns no results (meals is null)

Presenting recipes — use this format:
RECIPE: [name]
Category: [category] | Cuisine: [cuisine]

INGREDIENTS:
- [quantity] [ingredient]
...

INSTRUCTIONS:
1. [step]
...

Rules:
- Never invent recipes — only present what you find from tools
- Include ALL ingredients with exact quantities from the source
- If no results, suggest alternative search terms
```

---

## The Form and Shopping List

### Stage 2: Review Recipe (Form)

After the agent finds a recipe, the workflow **pauses** and shows a form. This is the **Wait** node in `On Form Submitted` mode — the same idea as Human-in-the-Loop, but with a proper web form.

| Field | Type | Purpose |
|-------|------|---------|
| **Recipe** | Custom HTML (read-only) | Displays the recipe the agent found |
| **Generate shopping list?** | Dropdown | `Yes, with price estimates` or `No, just the recipe` |

### How to Open the Form

1. Run the workflow (type in the chat)
2. The agent finds a recipe and the workflow pauses at the **Review Recipe** node
3. Click the **Review Recipe** node in the execution panel
4. Copy the **Test URL** from the output
5. Open that URL in your browser — you'll see the form
6. Choose your option and click **Continue**

### Stage 3: Shopping List (Optional)

If you chose "Yes", a **Basic LLM Chain** generates a shopping list with estimated prices. It is not an agent — it is a single, focused LLM call.

| | Agent | Basic LLM Chain |
|--|-------|----------------|
| **Use when** | Decisions needed, tools involved | Straightforward text in → text out |
| **This workflow** | Recipe search (which tool? what query?) | Shopping list (just format the recipe into a list) |

The Shopping List LLM runs **after** the Wait node. It cannot use `{{ $json.output }}` because `$json` now contains the form data. Instead, it references the agent's output directly:

```
{{ $('Recipe Search Agent').first().json.output }}
```

This expression says: "Go to the Recipe Search Agent node, get its first output item, and read the `output` field."

---

## Full Circle: From Fixed Chain to Smart Agent

Compare what you built in the Prompt Chaining section with this project:

| | Prompt Chaining (Ch. 4) | This Project |
|--|---------------------------|-------------|
| **Recipe source** | Hard-coded in the Input node | Fetched live from the internet |
| **Step order** | Fixed: Step 1 → Step 2 → Step 3 | Agent decides the order |
| **Shopping list** | Always generated | User chooses via form |
| **Error handling** | None — fails if input is bad | Fallback from TheMealDB to Google |
| **Memory** | None — single execution | Remembers conversation across turns |
| **Human control** | None — runs start to finish | Form lets user review and decide |
| **Input** | One hard-coded request | Interactive chat |

---

## Try It Yourself

### Challenge 1: Random Recipe Tool

Add a third tool to the agent using TheMealDB's random endpoint:

```
https://www.themealdb.com/api/json/v1/1/random.php
```

This takes no parameters — it returns one random recipe. Update the system message so the agent uses this tool when the user says "surprise me" or "pick something random."

**Done when:** saying "surprise me" triggers the random endpoint and returns a complete recipe in the form.

### Challenge 2: Category Filter Tool

Add a tool using TheMealDB's category filter:

```
https://www.themealdb.com/api/json/v1/1/filter.php?c={category}
```

Categories include: `Chicken`, `Seafood`, `Vegetarian`, `Dessert`, `Beef`, `Pasta`. The agent can use this to browse options before searching for a specific recipe.

**Done when:** asking "show me seafood options" returns a list from the Seafood category.

### Challenge 3: Real Prices with Spoonacular API

Replace LLM-estimated prices with real ingredient prices using the [Spoonacular API](https://spoonacular.com/food-api) (free tier: 50 points/day).

**Endpoints:**
- Search: `GET https://api.spoonacular.com/food/ingredients/search?query={name}&number=1&apiKey=KEY`
- Price: `GET https://api.spoonacular.com/food/ingredients/{id}/information?amount=1&unit=piece&apiKey=KEY`

The price comes back in US cents (`estimatedCost.value`). The free tier allows ~24 ingredient lookups per day.

**Tip:** Have the agent look up prices for only the 3-5 most expensive ingredients and estimate the rest — this keeps API usage low.

**Done when:** the shopping list includes at least 3 real prices from Spoonacular.

---

## Summary

| Concept | What You Learned |
|---------|------------------|
| **HTTP Request Tool** | Agents can call any web API using `$fromAI()` to let the AI fill in parameters |
| **Fallback strategy** | Use a reliable API first, fall back to a broader search if needed |
| **Chat Trigger + Memory** | Build interactive, multi-turn conversations |
| **Wait node with Form** | Pause the workflow and show a web form for human review |
| **IF branching** | Route the workflow based on user choices from the form |
| **`$('Node').first().json.field`** | Reference any previous node's output, even after a Wait |

**Key expressions:**
- `{{ $json.chatInput }}` — the user's message from Chat Trigger
- `{{ $json.output }}` — the agent's response (AI Agent field)
- `{{ $json.text }}` — a Basic LLM Chain's response
- `{{ $json["Generate shopping list?"] }}` — a form dropdown value
- `{{ $('Recipe Search Agent').first().json.output }}` — reference a specific node's output

**Docs:**
- [HTTP Request Tool](https://docs.n8n.io/integrations/builtin/cluster-nodes/sub-nodes/n8n-nodes-langchain.toolhttprequest/)
- [Chat Trigger](https://docs.n8n.io/integrations/builtin/cluster-nodes/root-nodes/n8n-nodes-langchain.chattrigger/)
- [Window Buffer Memory](https://docs.n8n.io/integrations/builtin/cluster-nodes/sub-nodes/n8n-nodes-langchain.memorybufferwindow/)
- [Wait node](https://docs.n8n.io/integrations/builtin/core-nodes/n8n-nodes-base.wait/)
- [TheMealDB API](https://www.themealdb.com/api.php)